#### Rating by movie title -> using dataframes on the top of RDD. the latest version of spark support dataframes an is faster than RDD

# Dataframes on the top RDD

# Query two -> step 1: average rating per movie id

# step 1: cache movie dataset in a momory and broadcast

- create two rdd and join them based on key


In [77]:
import collections
import time
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\Users\\afoto001.000\\spark'

In [78]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName("SparkApp").setMaster('local')
sc = pyspark.SparkContext(conf=conf) # sc is a spark context that creats an RDD object 
spark = SparkSession(sc)

In [79]:
start = time.time()
def parseLine(line):
    fields = line.split(';')
    movie_id = int(fields[1])
    movie_title = float(fields[2])
    return(movie_id,movie_title)
lines = sc.textFile("C:/Users/afoto001.000/Desktop/Abraham_edu_doc/UvA/Master/Big data/BigData/bigdata/data/ml-latest-small/ratings.csv")
rdd = lines.map(parseLine)

In [80]:
totalsById = rdd.mapValues(lambda x:(x,1)).reduceByKey(lambda x,y:(x[0] +y[0],x[1] + y[1]))
averagesById = totalsById.mapValues(lambda x: x[0]/x[1])
results = averagesById.collect()
for result in results:
    print(result)
end = time.time()
print("Time taken in seconds: " + str(end-start) + ' s')

(1, 3.9209302325581397)
(3, 3.2596153846153846)
(6, 3.946078431372549)
(47, 3.9753694581280787)
(50, 4.237745098039215)
(70, 3.5090909090909093)
(101, 3.782608695652174)
(110, 4.031645569620253)
(151, 3.5454545454545454)
(157, 2.8636363636363638)
(163, 3.5606060606060606)
(216, 3.326530612244898)
(223, 3.855769230769231)
(231, 3.0601503759398496)
(235, 3.6785714285714284)
(260, 4.231075697211155)
(296, 4.197068403908795)
(316, 3.375)
(333, 3.78)
(349, 3.6045454545454545)
(356, 4.164133738601824)
(362, 3.5294117647058822)
(367, 3.1847133757961785)
(423, 2.85)
(441, 3.9285714285714284)
(457, 3.9921052631578946)
(480, 3.75)
(500, 3.388888888888889)
(527, 4.225)
(543, 3.317073170731707)
(552, 3.262295081967213)
(553, 3.8153846153846156)
(590, 3.8353658536585367)
(592, 3.4285714285714284)
(593, 4.161290322580645)
(596, 3.45)
(608, 4.116022099447513)
(648, 3.537037037037037)
(661, 3.4489795918367347)
(673, 2.707547169811321)
(733, 3.6404958677685952)
(736, 3.321138211382114)
(780, 3.44554455

(2735, 2.71875)
(2794, 2.6333333333333333)
(3066, 3.9285714285714284)
(3270, 2.7916666666666665)
(3347, 4.166666666666667)
(3362, 3.9342105263157894)
(3766, 2.0833333333333335)
(3861, 2.6842105263157894)
(4220, 3.4)
(5093, 2.7777777777777777)
(5312, 2.857142857142857)
(6662, 3.4545454545454546)
(585, 2.736111111111111)
(2342, 4.0)
(3019, 4.15)
(82, 3.875)
(121, 4.0)
(199, 4.125)
(228, 2.3333333333333335)
(306, 4.21875)
(514, 3.4166666666666665)
(685, 4.0)
(3285, 3.0625)
(3968, 3.074074074074074)
(4890, 3.032258064516129)
(6708, 3.7205882352941178)
(6763, 2.4285714285714284)
(7173, 3.0416666666666665)
(7444, 3.1904761904761907)
(8981, 3.71875)
(54001, 3.8620689655172415)
(55280, 3.5625)
(56941, 3.090909090909091)
(59725, 2.409090909090909)
(61240, 3.911764705882353)
(65230, 3.142857142857143)
(71033, 3.857142857142857)
(85774, 3.75)
(86781, 4.4)
(87869, 3.4375)
(89759, 4.333333333333333)
(93840, 4.0227272727272725)
(96829, 4.3)
(97921, 3.716666666666667)
(100714, 3.5)
(101525, 3.5)
(103

(1829, 3.25)
(1897, 3.5)
(2132, 4.142857142857143)
(2166, 2.5)
(2331, 3.0)
(2475, 3.6666666666666665)
(2749, 4.0)
(2764, 4.0)
(2844, 2.6666666666666665)
(2969, 5.0)
(3099, 3.0714285714285716)
(3103, 2.75)
(3217, 3.5)
(3251, 3.7)
(3296, 4.071428571428571)
(3537, 3.3333333333333335)
(3546, 4.5)
(3831, 3.909090909090909)
(4003, 3.3333333333333335)
(4063, 2.5)
(4083, 4.0)
(4130, 1.8333333333333333)
(127198, 3.5833333333333335)
(141718, 5.0)
(167732, 3.5)
(487, 2.5)
(2772, 2.85)
(4792, 3.125)
(62383, 3.75)
(73, 3.4615384615384617)
(656, 3.0)
(46337, 2.25)
(66511, 4.25)
(85020, 3.375)
(95105, 3.388888888888889)
(99112, 3.4166666666666665)
(103984, 3.5)
(108689, 2.25)
(109846, 3.25)
(110718, 4.0)
(110730, 2.2857142857142856)
(112868, 4.5)
(114184, 3.5)
(114795, 3.5)
(117533, 3.875)
(117887, 4.0)
(120635, 2.7)
(127184, 4.0)
(128488, 3.0)
(130578, 4.0)
(131023, 4.5)
(131714, 3.0)
(135532, 3.3333333333333335)
(136562, 3.75)
(136666, 2.25)
(138210, 3.3)
(141004, 3.25)
(165551, 4.5)
(52319, 3.5714

(145080, 2.5)
(147002, 4.0)
(148424, 3.75)
(150696, 4.5)
(152065, 3.5)
(155774, 3.5)
(158842, 3.5)
(159976, 4.5)
(160684, 3.5)
(168612, 3.75)
(170355, 3.75)
(171495, 4.5)
(173197, 4.25)
(173291, 2.875)
(173317, 4.5)
(179135, 5.0)
(184245, 5.0)
(188675, 3.5)
(188833, 4.5)
(189381, 2.5)
(26555, 2.75)
(2882, 2.0)
(110127, 2.8)
(111443, 2.75)
(3884, 1.5)
(3899, 1.0)
(3912, 2.5)
(30, 3.0)
(80, 4.0)
(183, 3.5)
(2425, 4.0)
(2848, 4.0)
(2894, 4.0)
(4086, 3.5)
(4263, 4.0)
(4294, 4.0)
(4471, 1.5)
(4500, 4.0)
(4523, 3.5)
(4714, 2.0)
(4717, 4.0)
(4927, 3.0)
(5033, 2.5)
(5075, 3.6666666666666665)
(5980, 3.5)
(27866, 3.5)
(70451, 5.0)
(73344, 4.25)
(81257, 3.5)
(84240, 4.0)
(102194, 4.333333333333333)
(165101, 2.875)
(130482, 4.0)
(136850, 5.0)
(26528, 4.5)
(193609, 4.0)
(3420, 3.1666666666666665)
(5226, 2.25)
(7925, 4.25)
(26564, 3.5)
(80917, 3.5)
(92938, 1.6)
(93610, 3.0)
(97757, 4.0)
(3567, 5.0)
(3580, 3.5)
(30850, 3.75)
(47404, 4.25)
(59118, 3.5)
(189043, 2.5)
(189111, 3.0)
(190207, 1.5)
(190209

(34800, 1.5)
(34811, 2.5)
(36850, 3.75)
(39801, 2.75)
(40578, 3.0)
(41627, 3.5)
(42559, 3.0)
(42638, 2.5)
(42943, 2.5)
(42946, 4.0)
(43419, 1.0)
(43677, 2.0)
(43908, 3.0)
(44238, 2.5)
(44241, 2.0)
(44243, 0.5)
(44849, 2.5)
(45183, 3.0)
(45648, 2.5)
(45969, 2.5)
(46367, 2.0)
(46574, 2.5)
(47721, 3.5)
(47793, 2.5)
(49110, 2.0)
(51545, 2.5)
(53022, 3.75)
(53143, 2.0)
(53161, 3.0)
(53450, 1.5)
(54274, 0.5)
(54908, 2.0)
(55036, 2.5)
(55061, 1.0)
(55205, 2.5)
(55207, 3.0)
(56022, 2.0)
(57499, 3.0)
(58306, 2.5)
(59016, 2.5)
(59336, 3.0)
(59947, 2.0)
(60365, 1.5)
(60803, 2.5)
(61073, 2.0)
(62662, 3.0)
(62970, 2.25)
(66915, 2.0)
(67193, 2.5)
(68552, 2.0)
(68650, 2.0)
(69131, 2.5)
(69227, 1.0)
(70703, 2.0)
(71327, 3.5)
(71484, 2.5)
(71550, 1.0)
(71810, 0.5)
(72479, 3.5)
(73569, 3.75)
(73808, 3.0)
(74095, 2.5)
(74696, 3.0)
(75416, 1.5)
(75947, 2.0)
(76030, 0.5)
(77201, 2.5)
(77709, 2.5)
(78959, 3.0)
(79895, 2.5)
(80083, 4.0)
(81791, 3.0)
(82534, 2.5)
(83086, 2.5)
(83796, 2.5)
(84523, 3.0)
(85438,

In [64]:
sc.stop()

# Step 2:

In [81]:
start = time.time()
def loadMovieNames():
    movieNames = {}
    # CHANGE THIS TO THE PATH TO YOUR u.ITEM FILE:
    with open("C:/Users/afoto001.000/Desktop/Abraham_edu_doc/UvA/Master/Big data/BigData/bigdata/data/ml-latest-small/movies.csv", "r", encoding='ISO-8859-1', errors='ignore') as f:
        for line in f:
            fields = line.split(';')
            print(fields[0], fields[1])
            movieNames[int(fields[0])] = fields[1]
    return movieNames

In [82]:
loadMovieNames()

1 Toy Story (1995)
2 Jumanji (1995)
3 Grumpier Old Men (1995)
4 Waiting to Exhale (1995)
5 Father of the Bride Part II (1995)
6 Heat (1995)
7 Sabrina (1995)
8 Tom and Huck (1995)
9 Sudden Death (1995)
10 GoldenEye (1995)
11 American President, The (1995)
12 Dracula: Dead and Loving It (1995)
13 Balto (1995)
14 Nixon (1995)
15 Cutthroat Island (1995)
16 Casino (1995)
17 Sense and Sensibility (1995)
18 Four Rooms (1995)
19 Ace Ventura: When Nature Calls (1995)
20 Money Train (1995)
21 Get Shorty (1995)
22 Copycat (1995)
23 Assassins (1995)
24 Powder (1995)
25 Leaving Las Vegas (1995)
26 Othello (1995)
27 Now and Then (1995)
28 Persuasion (1995)
29 City of Lost Children, The (CitÃ© des enfants perdus, La) (1995)
30 Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
31 Dangerous Minds (1995)
32 Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
34 Babe (1995)
36 Dead Man Walking (1995)
38 It Takes Two (1995)
39 Clueless (1995)
40 Cry, the Beloved Country (1995)
41 Richard III (1995)
42 Dead Presi

1626 Fire Down Below (1997)
1627 U Turn (1997)
1629 MatchMaker, The (1997)
1631 Assignment, The (1997)
1633 Ulee's Gold (1997)
1635 Ice Storm, The (1997)
1639 Chasing Amy (1997)
1641 Full Monty, The (1997)
1642 Indian Summer (a.k.a. Alive & Kicking) (1996)
1643 Mrs. Brown (a.k.a. Her Majesty, Mrs. Brown) (1997)
1644 I Know What You Did Last Summer (1997)
1645 The Devil's Advocate (1997)
1646 RocketMan (a.k.a. Rocket Man) (1997)
1647 Playing God (1997)
1648 House of Yes, The (1997)
1649 Fast, Cheap & Out of Control (1997)
1650 Washington Square (1997)
1652 Year of the Horse (1997)
1653 Gattaca (1997)
1654 FairyTale: A True Story (1997)
1655 Phantoms (1998)
1656 Swept from the Sea (1997)
1658 Life Less Ordinary, A (1997)
1659 Hurricane Streets (1997)
1660 Eve's Bayou (1997)
1661 Switchback (1997)
1662 Gang Related (1997)
1663 Stripes (1981)
1665 Bean (1997)
1667 Mad City (1997)
1670 Welcome to Sarajevo (1997)
1671 Deceiver (1997)
1672 Rainmaker, The (1997)
1673 Boogie Nights (1997)
1674 

3104 Midnight Run (1988)
3105 Awakenings (1990)
3106 Come See the Paradise (1990)
3107 Backdraft (1991)
3108 Fisher King, The (1991)
3109 River, The (1984)
3111 Places in the Heart (1984)
3112 'night Mother (1986)
3113 End of Days (1999)
3114 Toy Story 2 (1999)
3115 Flawless (1999)
3117 Ride with the Devil (1999)
3120 Distinguished Gentleman, The (1992)
3125 End of the Affair, The (1999)
3127 Holy Smoke (1999)
3129 Sweet and Lowdown (1999)
3130 Bonfire of the Vanities (1990)
3132 Daddy Long Legs (1919)
3134 Grand Illusion (La grande illusion) (1937)
3135 Great Santini, The (1979)
3138 Stealing Home (1988)
3141 Two Jakes, The (1990)
3142 U2: Rattle and Hum (1988)
3143 Hell in the Pacific (1968)
3145 Cradle Will Rock (1999)
3146 Deuce Bigalow: Male Gigolo (1999)
3147 Green Mile, The (1999)
3148 Cider House Rules, The (1999)
3150 War Zone, The (1999)
3152 Last Picture Show, The (1971)
3153 7th Voyage of Sinbad, The (1958)
3155 Anna and the King (1999)
3156 Bicentennial Man (1999)
3157 Stu

4871 Focus (2001)
4873 Waking Life (2001)
4874 K-PAX (2001)
4876 Thirteen Ghosts (a.k.a. Thir13en Ghosts) (2001)
4877 Better Than Sex (2000)
4878 Donnie Darko (2001)
4879 High Heels and Low Lifes (2001)
4880 Life as a House (2001)
4881 Man Who Wasn't There, The (2001)
4883 Town is Quiet, The (Ville est tranquille, La) (2000)
4885 Domestic Disturbance (2001)
4886 Monsters, Inc. (2001)
4887 One, The (2001)
4888 Tape (2001)
4889 Heist (2001)
4890 Shallow Hal (2001)
4893 When a Stranger Calls (1979)
4896 Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
4898 Novocaine (2001)
4899 Black Knight (2001)
4900 Out Cold (2001)
4901 Spy Game (2001)
4902 Devil's Backbone, The (Espinazo del diablo, El) (2001)
4903 In the Bedroom (2001)
4909 Incredible Shrinking Woman, The (1981)
4911 Jabberwocky (1977)
4912 Funny Girl (1968)
4914 Breathless (Ã bout de souffle) (1960)
4915 Beastmaster, The (1982)
4916 Midway (1976)
4917 MacArthur (1977)
4919 Subway (1985)

7457 Breakin' All the Rules (2004)
7458 Troy (2004)
7459 Carandiru (2003)
7460 Coffee and Cigarettes (2003)
7477 Eye See You (D-Tox) (2002)
7479 Watch on the Rhine (1943)
7481 Enemy Mine (1985)
7482 Enter the Dragon (1973)
7486 Happy Together (a.k.a. Buenos Aires Affair) (Chun gwong cha sit) (1997)
7487 Henry & June (1990)
7492 Martin (1977)
7493 Three Faces of Eve, The (1957)
7541 100 Girls (2000)
7560 Fail-Safe (1964)
7561 Paperhouse (1988)
7562 Dobermann (1997)
7564 Kwaidan (Kaidan) (1964)
7566 28 Up (1985)
7569 You Only Live Twice (1967)
7570 Octopussy (1983)
7572 Wit (2001)
7573 Never Say Never Again (1983)
7579 Pride and Prejudice (1940)
7581 Fountainhead, The (1949)
7584 Woman of the Year (1942)
7587 SamouraÃ¯, Le (Godson, The) (1967)
7614 Oklahoma! (1955)
7615 Desert Hearts (1985)
7616 Body Double (1984)
7618 Chaplin (1992)
7619 Miracle Worker, The (1962)
7620 Monster in a Box (1992)
7624 School Ties (1992)
7636 Raising Cain (1992)
7646 Rose Red (2002)
7647 Noises Off... (1992)

44195 Thank You for Smoking (2006)
44197 Find Me Guilty (2006)
44199 Inside Man (2006)
44204 Tsotsi (2005)
44225 Aquamarine (2006)
44238 Leprechaun 2 (1994)
44241 Leprechaun 3 (1995)
44243 Leprechaun 4: In Space (1997)
44301 Lights in the Dusk (Laitakaupungin valot) (2006)
44397 Hills Have Eyes, The (2006)
44399 Shaggy Dog, The (2006)
44511 Unknown White Male (2005)
44555 Lives of Others, The (Das leben der Anderen) (2006)
44597 Youth of the Beast (Yaju no seishun) (1963)
44613 Take the Lead (2006)
44633 Devil and Daniel Johnston, The (2005)
44657 Mouchette (1967)
44665 Lucky Number Slevin (2006)
44694 Volver (2006)
44709 Akeelah and the Bee (2006)
44719 Brainstorm (2001)
44731 Stay Alive (2006)
44759 Basic Instinct 2 (2006)
44761 Brick (2005)
44773 Dead Hate the Living!, The (2000)
44777 Evil Aliens (2005)
44788 This Film Is Not Yet Rated (2006)
44828 Slither (2006)
44840 Benchwarmers, The (2006)
44849 Renaissance (2006)
44851 Go for Zucker! (Alles auf Zucker!) (2004)
44864 Friends wi

65359 Earthsea (Legend of Earthsea) (2004)
65514 Ip Man (2008)
65577 Tale of Despereaux, The (2008)
65585 Bride Wars (2009)
65588 Gamers, The: Dorkness Rising (2008)
65596 Mesrine: Killer Instinct (L'instinct de mort) (2008)
65601 My Bloody Valentine 3-D (2009)
65631 Battle in Seattle (2007)
65642 Timecrimes (CronocrÃ­menes, Los) (2007)
65651 Fire and Ice (2008)
65682 Underworld: Rise of the Lycans (2009)
65685 Inkheart (2008)
65738 Revenge of the Nerds III: The Next Generation (1992)
65740 Revenge of the Nerds IV: Nerds in Love (1994)
65802 Paul Blart: Mall Cop (2009)
65810 Notorious (2009)
65882 Uninvited, The (2009)
65982 Outlander (2008)
66090 Eden Lake (2008)
66097 Coraline (2009)
66171 Push (2009)
66198 International, The (2009)
66203 He's Just Not That Into You (2009)
66240 Dead Like Me: Life After Death (2009)
66297 Futurama: Into the Wild Green Yonder (2009)
66310 FrontiÃ¨re(s) (2007)
66320 11th Hour, The (2007)
66335 Afro Samurai: Resurrection (2009)
66371 Departures (Okuribi

95858 For the Birds (2000)
95873 Ruby Sparks (2012)
95875 Total Recall (2012)
95939 Angels' Share, The (2012)
95949 Immature, The (Immaturi) (2011)
95965 Dragon Ball Z: Bardock - The Father of Goku (Doragon bÃ´ru Z: Tatta hitori no saishuu kessen - Furiiza ni itonda Z senshi Kakarotto no chichi) (1990)
96004 Dragon Ball Z: The History of Trunks (Doragon bÃ´ru Z: ZetsubÃ´ e no hankÃ´!! Nokosareta chÃ´ senshi - Gohan to Torankusu) (1993)
96007 Dragon Ball GT: A Hero's Legacy (Doragon bÃ´ru GT: GokÃ» gaiden! YÃ»ki no akashi wa sÃ»-shin-chÃ») (1997)
96020 Sidewalls (Medianeras) (2011)
96079 Skyfall (2012)
96084 Hands Over the City (Le mani sulla cittÃ ) (1963)
96110 Campaign, The (2012)
96114 Brake (2012)
96121 Hope Springs (2012)
96150 Queen of Versailles, The (2012)
96281 ParaNorman (2012)
96283 Diary of a Wimpy Kid: Dog Days (2012)
96373 Broken (2012)
96411 6 Days to Air: The Making of South Park (2011)
96417 Premium Rush (2012)
96430 Odd Life of Timothy Green, The (2012)
96432 Lawless 

136556 Kung Fu Panda: Secrets of the Masters (2011)
136562 Steve Jobs (2015)
136564 Macbeth (2015)
136598 Vacation (2015)
136602 Creep (2014)
136654 The Face of an Angel (2015)
136664 Wild Horses (2015)
136666 Search Party (2014)
136778 The Squeeze (2015)
136786 Careful What You Wish For (2015)
136800 Robot Overlords (2014)
136816 Bad Asses on the Bayou (2015)
136834 The Eye: Infinity (2005)
136838 Kiss me Kismet (2006)
136840 Da geht noch was! (2013)
136850 Villain (1971)
136859 The Lovers (2015)
136864 Batman v Superman: Dawn of Justice (2016)
136912 God Loves Caviar (2012)
136958 Mortuary (1983)
137218 April Morning (1988)
137337 Amy (2015)
137345 That Demon Within (2014)
137517 Carnival Magic (1981)
137595 Magic Mike XXL (2015)
137857 The Jungle Book (2016)
137859 Dragon Ball Z Gaiden: The Plot to Destroy the Saiyans (1993)
137863 Dragon Ball Z: Resurrection of F (2015)
138036 The Man from U.N.C.L.E. (2015)
138186 Sorrow (2015)
138204 7 Days in Hell (2015)
138208 The Walk (2015)
13

{1: 'Toy Story (1995)',
 2: 'Jumanji (1995)',
 3: 'Grumpier Old Men (1995)',
 4: 'Waiting to Exhale (1995)',
 5: 'Father of the Bride Part II (1995)',
 6: 'Heat (1995)',
 7: 'Sabrina (1995)',
 8: 'Tom and Huck (1995)',
 9: 'Sudden Death (1995)',
 10: 'GoldenEye (1995)',
 11: 'American President, The (1995)',
 12: 'Dracula: Dead and Loving It (1995)',
 13: 'Balto (1995)',
 14: 'Nixon (1995)',
 15: 'Cutthroat Island (1995)',
 16: 'Casino (1995)',
 17: 'Sense and Sensibility (1995)',
 18: 'Four Rooms (1995)',
 19: 'Ace Ventura: When Nature Calls (1995)',
 20: 'Money Train (1995)',
 21: 'Get Shorty (1995)',
 22: 'Copycat (1995)',
 23: 'Assassins (1995)',
 24: 'Powder (1995)',
 25: 'Leaving Las Vegas (1995)',
 26: 'Othello (1995)',
 27: 'Now and Then (1995)',
 28: 'Persuasion (1995)',
 29: 'City of Lost Children, The (CitÃ© des enfants perdus, La) (1995)',
 30: 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 31: 'Dangerous Minds (1995)',
 32: 'Twelve Monkeys (a.k.a. 12 Monkeys) (19

In [83]:
my_dict_bc = sc.broadcast(loadMovieNames())

1 Toy Story (1995)
2 Jumanji (1995)
3 Grumpier Old Men (1995)
4 Waiting to Exhale (1995)
5 Father of the Bride Part II (1995)
6 Heat (1995)
7 Sabrina (1995)
8 Tom and Huck (1995)
9 Sudden Death (1995)
10 GoldenEye (1995)
11 American President, The (1995)
12 Dracula: Dead and Loving It (1995)
13 Balto (1995)
14 Nixon (1995)
15 Cutthroat Island (1995)
16 Casino (1995)
17 Sense and Sensibility (1995)
18 Four Rooms (1995)
19 Ace Ventura: When Nature Calls (1995)
20 Money Train (1995)
21 Get Shorty (1995)
22 Copycat (1995)
23 Assassins (1995)
24 Powder (1995)
25 Leaving Las Vegas (1995)
26 Othello (1995)
27 Now and Then (1995)
28 Persuasion (1995)
29 City of Lost Children, The (CitÃ© des enfants perdus, La) (1995)
30 Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
31 Dangerous Minds (1995)
32 Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
34 Babe (1995)
36 Dead Man Walking (1995)
38 It Takes Two (1995)
39 Clueless (1995)
40 Cry, the Beloved Country (1995)
41 Richard III (1995)
42 Dead Presi

1971 Nightmare on Elm Street 4: The Dream Master, A (1988)
1972 Nightmare on Elm Street 5: The Dream Child, A (1989)
1973 Freddy's Dead: The Final Nightmare (Nightmare on Elm Street Part 6: Freddy's Dead, A) (1991)
1974 Friday the 13th (1980)
1975 Friday the 13th Part 2 (1981)
1976 Friday the 13th Part 3: 3D (1982)
1977 Friday the 13th Part IV: The Final Chapter (1984)
1978 Friday the 13th Part V: A New Beginning (1985)
1979 Friday the 13th Part VI: Jason Lives (1986)
1980 Friday the 13th Part VII: The New Blood (1988)
1981 Friday the 13th Part VIII: Jason Takes Manhattan (1989)
1982 Halloween (1978)
1983 Halloween II (1981)
1984 Halloween III: Season of the Witch (1982)
1985 Halloween 4: The Return of Michael Myers (1988)
1986 Halloween 5: The Revenge of Michael Myers (1989)
1987 Prom Night (1980)
1990 Prom Night IV: Deliver Us From Evil (1992)
1991 Child's Play (1988)
1992 Child's Play 2 (1990)
1993 Child's Play 3 (1991)
1994 Poltergeist (1982)
1995 Poltergeist II: The Other Side (19

3296 To Sir with Love (1967)
3298 Boiler Room (2000)
3299 Hanging Up (2000)
3300 Pitch Black (2000)
3301 Whole Nine Yards, The (2000)
3302 Beautiful People (1999)
3303 Black Tar Heroin: The Dark End of the Street (2000)
3306 Circus, The (1928)
3307 City Lights (1931)
3308 Flamingo Kid, The (1984)
3310 Kid, The (1921)
3313 Class Reunion (1982)
3315 Happy Go Lovely (1951)
3316 Reindeer Games (2000)
3317 Wonder Boys (2000)
3323 Chain of Fools (2000)
3324 Drowning Mona (2000)
3325 Next Best Thing, The (2000)
3326 What Planet Are You From? (2000)
3327 Beyond the Mat (1999)
3328 Ghost Dog: The Way of the Samurai (1999)
3329 The Year My Voice Broke (1987)
3330 Splendor in the Grass (1961)
3331 My Tutor (1983)
3334 Key Largo (1948)
3338 For All Mankind (1989)
3341 Born Yesterday (1950)
3342 Birdy (1984)
3344 Blood Feast (1963)
3345 Charlie, the Lonesome Cougar (1967)
3347 Never Cry Wolf (1983)
3350 Raisin in the Sun, A (1961)
3353 Closer You Get, The (2000)
3354 Mission to Mars (2000)
3355 Nin

5139 Bad News Bears, The (1976)
5146 Vampire Hunter D: Bloodlust (Banpaia hantÃ¢ D) (2000)
5147 Wild Strawberries (SmultronstÃ¤llet) (1957)
5151 40 Days and 40 Nights (2002)
5152 We Were Soldiers (2002)
5155 Big Bird Cage, The (1972)
5156 Big Doll House, The (1971)
5159 Ferngully: The Last Rainforest (1992)
5165 Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979)
5168 Royal Wedding (1951)
5170 All About the Benjamins (2002)
5171 Time Machine, The (2002)
5177 Magnificent Ambersons, The (1942)
5178 Fun (1994)
5181 Hangar 18 (1980)
5187 Hopscotch (1980)
5189 Humanoids from the Deep (1980)
5197 Little Darlings (1980)
5198 Long Good Friday, The (1980)
5202 Mon oncle d'AmÃ©rique (1980)
5205 Motel Hell (1980)
5209 Ffolkes (1979)
5212 Octagon, The (1980)
5213 Oh, God! Book II (1980)
5214 Oh, God! (1977)
5218 Ice Age (2002)
5219 Resident Evil (2002)
5220 Showtime (2002)
5221 Harrison's Flowers (2000)
5222 Kissing Jessica Stein (2001)
5224 Promises (2001)
5225 And Your Mother Too (Y

7349 Broken Wings (Knafayim Shvurot) (2002)
7352 Wilbur Wants to Kill Himself (2002)
7354 Mad Dog and Glory (1993)
7357 Peyton Place (1957)
7360 Dawn of the Dead (2004)
7361 Eternal Sunshine of the Spotless Mind (2004)
7362 Taking Lives (2004)
7364 Intermission (2003)
7366 Jersey Girl (2004)
7367 Ladykillers, The (2004)
7368 Never Die Alone (2004)
7369 Scooby-Doo 2: Monsters Unleashed (2004)
7371 Dogville (2003)
7372 Ned Kelly (2003)
7373 Hellboy (2004)
7375 Prince & Me, The (2004)
7376 Walking Tall (2004)
7377 United States of Leland, The (2003)
7379 The Alamo (2004)
7380 Ella Enchanted (2004)
7381 Whole Ten Yards, The (2004)
7382 I'm Not Scared (Io non ho paura) (2003)
7386 Ten Commandments, The (1956)
7387 Dawn of the Dead (1978)
7390 Prey for Rock & Roll (2003)
7394 Those Magnificent Men in Their Flying Machines (1965)
7395 Cheaper by the Dozen (1950)
7396 Scenes From a Marriage (Scener ur ett Ã¤ktenskap) (1973)
7411 Munchies (1987)
7414 Going in Style (1979)
7419 After Hours (1985

38159 Short Film About Love, A (KrÃ³tki film o milosci) (1988)
38164 All This, and Heaven Too (1940)
38198 Darwin's Nightmare (2004)
38294 Beowulf & Grendel (2005)
38304 No Direction Home: Bob Dylan (2005)
38388 Goal! The Dream Begins (Goal!) (2005)
38583 Wraith, The (1986)
38798 In Her Shoes (2005)
38886 Squid and the Whale, The (2005)
38992 Two for the Money (2005)
39183 Brokeback Mountain (2005)
39231 Elizabethtown (2005)
39234 North Country (2005)
39292 Good Night, and Good Luck. (2005)
39307 Dreamer: Inspired by a True Story (2005)
39381 Proposition, The (2005)
39400 Fog, The (2005)
39414 Shopgirl (2005)
39427 Stay (2005)
39435 Legend of Zorro, The (2005)
39444 Weather Man, The (2005)
39446 Saw II (2005)
39449 Prime (2005)
39516 Don't Move (Non ti muovere) (2004)
39715 American Pie Presents: Band Camp (American Pie 4: Band Camp) (2005)
39801 Great Yokai War, The (YÃ´kai daisensÃ´) (2005)
39869 Manderlay (2005)
40148 Revolver (2005)
40278 Jarhead (2005)
40339 Chicken Little (2005)


80906 Inside Job (2010)
80917 Monsters (2010)
80969 Never Let Me Go (2010)
81018 Illusionist, The (L'illusionniste) (2010)
81087 Luck by Chance (2009)
81132 Rubber (2010)
81156 Jackass 3D (2010)
81158 Restrepo (2010)
81191 Waiting for 'Superman' (2010)
81229 Red (2010)
81257 In a Better World (HÃ¦vnen) (2010)
81383 Heartbreaker (L'Arnacoeur) (2010)
81417 Paranormal Activity 2 (2010)
81456 Heartbeats (Les amours imaginaires) (2010)
81512 Hereafter (2010)
81520 Undisputed III: Redemption (2010)
81535 Saw VII 3D - The Final Chapter (2010)
81537 Due Date (2010)
81562 127 Hours (2010)
81564 Megamind (2010)
81591 Black Swan (2010)
81681 I Shot Jesse James (1949)
81782 Unstoppable (2010)
81784 Morning Glory (2010)
81786 Certified Copy (Copie conforme) (2010)
81788 Next Three Days, The (2010)
81791 Somewhere (2010)
81819 Biutiful (2010)
81831 First Beautiful Thing, The (La prima cosa bella) (2010)
81834 Harry Potter and the Deathly Hallows: Part 1 (2010)
81845 King's Speech, The (2010)
81847 T

117887 Paddington (2014)
117895 Maze Runner: Scorch Trials (2015)
117922 Ice Age: A Mammoth Christmas (2011)
118082 The Voices (2014)
118166 Courier (1987)
118198 The Green Prince (2014)
118248 Dying of the Light (2014)
118270 Hellbenders (2012)
118290 Omega Doom (1996)
118326 By the Gun (2014)
118354 Kill the Messenger (2014)
118512 Bring It On: In It To Win It (2007)
118530 Fans (1999)
118572 The Mule (2014)
118696 The Hobbit: The Battle of the Five Armies (2014)
118700 Selma (2014)
118702 Unbroken (2014)
118706 Black Sea (2015)
118784 Good Copy Bad Copy (2007)
118814 Playing It Cool (2014)
118834 National Lampoon's Bag Boy (2007)
118862 Closer to the Moon (2013)
118880 Girl Walks Home Alone at Night, A (2014)
118888 Dave Chappelle: For What it's Worth (2004)
118894 Scooby-Doo! Abracadabra-Doo (2010)
118896 Mommy (2014)
118900 Wild (2014)
118924 Top Five (2014)
118930 Bill Burr: I'm Sorry You Feel That Way (2014)
118985 Big Eyes (2014)
118997 Into the Woods (2014)
119068 Men, Women &

In [104]:
def my_func(movieId):
    return (movieId,my_dict_bc.value[movieId])

In [108]:
final_result = averagesById.map(lambda x: my_func(x[0]))


[(1, 'Toy Story (1995)'),
 (3, 'Grumpier Old Men (1995)'),
 (6, 'Heat (1995)'),
 (47, 'Seven (a.k.a. Se7en) (1995)'),
 (50, 'Usual Suspects, The (1995)'),
 (70, 'From Dusk Till Dawn (1996)'),
 (101, 'Bottle Rocket (1996)'),
 (110, 'Braveheart (1995)'),
 (151, 'Rob Roy (1995)'),
 (157, 'Canadian Bacon (1995)'),
 (163, 'Desperado (1995)'),
 (216, 'Billy Madison (1995)'),
 (223, 'Clerks (1994)'),
 (231, 'Dumb & Dumber (Dumb and Dumber) (1994)'),
 (235, 'Ed Wood (1994)'),
 (260, 'Star Wars: Episode IV - A New Hope (1977)'),
 (296, 'Pulp Fiction (1994)'),
 (316, 'Stargate (1994)'),
 (333, 'Tommy Boy (1995)'),
 (349, 'Clear and Present Danger (1994)'),
 (356, 'Forrest Gump (1994)'),
 (362, 'Jungle Book, The (1994)'),
 (367, 'Mask, The (1994)'),
 (423, 'Blown Away (1994)'),
 (441, 'Dazed and Confused (1993)'),
 (457, 'Fugitive, The (1993)'),
 (480, 'Jurassic Park (1993)'),
 (500, 'Mrs. Doubtfire (1993)'),
 (527, "Schindler's List (1993)"),
 (543, 'So I Married an Axe Murderer (1993)'),
 (552,

In [112]:
r = final_result.join(averagesById).collect()

In [113]:
print(r)

[(6, ('Heat (1995)', 3.946078431372549)), (50, ('Usual Suspects, The (1995)', 4.237745098039215)), (70, ('From Dusk Till Dawn (1996)', 3.5090909090909093)), (110, ('Braveheart (1995)', 4.031645569620253)), (216, ('Billy Madison (1995)', 3.326530612244898)), (260, ('Star Wars: Episode IV - A New Hope (1977)', 4.231075697211155)), (296, ('Pulp Fiction (1994)', 4.197068403908795)), (316, ('Stargate (1994)', 3.375)), (356, ('Forrest Gump (1994)', 4.164133738601824)), (362, ('Jungle Book, The (1994)', 3.5294117647058822)), (480, ('Jurassic Park (1993)', 3.75)), (500, ('Mrs. Doubtfire (1993)', 3.388888888888889)), (552, ('Three Musketeers, The (1993)', 3.262295081967213)), (590, ('Dances with Wolves (1990)', 3.8353658536585367)), (592, ('Batman (1989)', 3.4285714285714284)), (596, ('Pinocchio (1940)', 3.45)), (608, ('Fargo (1996)', 4.116022099447513)), (648, ('Mission: Impossible (1996)', 3.537037037037037)), (736, ('Twister (1996)', 3.321138211382114)), (780, ('Independence Day (a.k.a. ID4)

In [122]:
for key,value in r:
    for x in value:
        print(x)

Heat (1995)
3.946078431372549
Usual Suspects, The (1995)
4.237745098039215
From Dusk Till Dawn (1996)
3.5090909090909093
Braveheart (1995)
4.031645569620253
Billy Madison (1995)
3.326530612244898
Star Wars: Episode IV - A New Hope (1977)
4.231075697211155
Pulp Fiction (1994)
4.197068403908795
Stargate (1994)
3.375
Forrest Gump (1994)
4.164133738601824
Jungle Book, The (1994)
3.5294117647058822
Jurassic Park (1993)
3.75
Mrs. Doubtfire (1993)
3.388888888888889
Three Musketeers, The (1993)
3.262295081967213
Dances with Wolves (1990)
3.8353658536585367
Batman (1989)
3.4285714285714284
Pinocchio (1940)
3.45
Fargo (1996)
4.116022099447513
Mission: Impossible (1996)
3.537037037037037
Twister (1996)
3.321138211382114
Independence Day (a.k.a. ID4) (1996)
3.4455445544554455
She's the One (1996)
3.25
Adventures of Robin Hood, The (1938)
4.0
Mr. Smith Goes to Washington (1939)
4.083333333333333
Three Caballeros, The (1945)
3.6666666666666665
Pete's Dragon (1977)
2.7666666666666666
Alice in Wonderl

3.6666666666666665
Maltese Falcon, The (a.k.a. Dangerous Female) (1931)
3.6176470588235294
Blind Swordsman: Zatoichi, The (ZatÃ´ichi) (2003)
3.9583333333333335
Virgin Suicides, The (1999)
3.769230769230769
The Count of Monte Cristo (2002)
3.828125
Constantine (2005)
3.4714285714285715
40-Year-Old Virgin, The (2005)
3.5472972972972974
Illusionist, The (2006)
3.7790697674418605
Cloverfield (2008)
3.3518518518518516
Pineapple Express (2008)
3.6774193548387095
Tropic Thunder (2008)
3.5
Midnight in Paris (2011)
3.56
Man Up (2015)
3.75
Avengers: Infinity War - Part I (2018)
4.0
Thor: Ragnarok (2017)
4.025
Far From Home: The Adventures of Yellow Dog (1995)
4.0
Alaska (1996)
3.5
Great Escape, The (1963)
4.127906976744186
Air Bud (1997)
2.375
Barefoot Executive, The (1971)
2.0
American Tail: Fievel Goes West, An (1991)
2.7
Beneath the Planet of the Apes (1970)
2.7777777777777777
Conquest of the Planet of the Apes (1972)
2.7916666666666665
Thumbelina (1994)
2.0
Adventures of Elmo in Grouchland, 

4.0
Witchfinder General (Conquerer Worm, The) (1968)
2.0
Warriors, The (1979)
3.9375
Getaway, The (1972)
4.0
Purple Rain (1984)
2.7
Robot Carnival (Roboto kÃ¢nibauru) (1987)
4.0
Outsiders, The (1983)
3.71875
Miracle on 34th Street (1947)
4.0588235294117645
Across the Universe (2007)
4.045454545454546
Kull the Conqueror (1997)
2.3333333333333335
Clay Pigeons (1998)
3.0
eXistenZ (1999)
3.8636363636363638
Porky's Revenge (1985)
2.0
Less Than Zero (1987)
3.5
Bubble Boy (2001)
1.8888888888888888
Ocean's Eleven (a.k.a. Ocean's 11) (1960)
3.6363636363636362
Metropolis (2001)
3.9615384615384617
Smokey and the Bandit II (1980)
2.125
Roger Dodger (2002)
3.6666666666666665
Better Luck Tomorrow (2002)
3.875
All the King's Men (1949)
4.25
Windtalkers (2002)
2.875
Terms of Endearment (1983)
3.7954545454545454
About Last Night... (1986)
3.0
Goodbye Girl, The (1977)
3.7083333333333335
Importance of Being Earnest, The (2002)
3.85
Paris, I Love You (Paris, je t'aime) (2006)
2.9375
First Sunday (2008)
3.

4.0
Revenge of the Nerds IV: Nerds in Love (1994)
4.0
Uninvited, The (2009)
3.5
Marty (1955)
3.8333333333333335
Suddenly, Last Summer (1959)
3.5
Sweet Smell of Success (1957)
4.125
Requiem for a Heavyweight (1962)
4.0
Long, Hot Summer, The (1958)
4.0
Taking Sides (2001)
3.5
Ox-Bow Incident, The (1943)
4.5
Random Harvest (1942)
4.0
Splendor in the Grass (1961)
4.25
Car Wash (1976)
3.75
This Property is Condemned (1966)
4.5
Cry, the Beloved Country (1995)
4.25
Alphaville (Alphaville, une Ã©trange aventure de Lemmy Caution) (1965)
4.125
Fury, The (1978)
3.0
Wicker Man, The (1973)
3.9166666666666665
Reform School Girls (1986)
5.0
Manhunter (1986)
3.9166666666666665
Two Much (1995)
3.0
Alvarez Kelly (1966)
3.0
Restrepo (2010)
3.1666666666666665
Blackfish (2013)
3.0
Ace in the Hole (Big Carnival, The) (1951)
3.0
Center of the World, The (2001)
3.6666666666666665
Jesus of Montreal (JÃ©sus de MontrÃ©al) (1989)
4.5
Marat/Sade (1966)
4.5
Mon oncle d'AmÃ©rique (1980)
3.25
Return of the Secaucus 7

2.0
Stoked: The Rise and Fall of Gator (2002)
2.0
Frankenstein Unbound (1990)
3.0
While the City Sleeps (1956)
0.5
Cincinnati Kid, The (1965)
0.5
Code 46 (2003)
4.5
Camera Buff (Amator) (1979)
3.0
Kung Fu Panda: Secrets of the Masters (2011)
5.0
Crimes of the Heart (1986)
4.0
Love Is a Many-Splendored Thing (1955)
4.0
Shootist, The (1976)
3.5
New York Cop (NyÃ» YÃ´ku no koppu) (1993)
4.5
Year of the Horse (1997)
4.5
Knock Off (1998)
5.0
Double Impact (1991)
3.25
Let It Snow (1999)
4.5
Opera (1987)
3.0
Tenebre (1982)
2.25
Hard Way, The (1991)
3.25
Revenge (1990)
4.0
Collector, The (1965)
0.5
Baxter (1989)
0.5
Sniper (1993)
3.0
Delta Force 2 (Delta Force 2: The Colombian Connection) (1990)
4.0
Better Than Chocolate (1999)
3.0
Maurice (1987)
4.0
Big Eden (2000)
4.5
Latter Days (2003)
3.5
If These Walls Could Talk 2 (2000)
4.0
Imagine Me & You (2005)
3.5
Itty Bitty Titty Committee (2007)
3.5
Prayers for Bobby (2009)
3.5
G.B.F. (2013)
3.0
White Frog (2012)
4.0
Cloudburst (2011)
4.0
Girltras

3.0
The Hundred-Foot Journey (2014)
4.5
Kingsman: The Secret Service (2015)
3.986111111111111
Magic Mike XXL (2015)
4.0
Program, The (1993)
3.0
Searching for Bobby Fischer (1993)
3.875
Under Siege (1992)
2.8541666666666665
Jackal, The (1997)
3.2222222222222223
Amistad (1997)
3.5
Titanic (1997)
3.414285714285714
Jane Austen's Mafia! (1998)
1.8333333333333333
Shine (1996)
4.0
Beavis and Butt-Head Do America (1996)
2.935483870967742
Karate Kid, Part II, The (1986)
2.7758620689655173
She's All That (1999)
2.8448275862068964
Never Been Kissed (1999)
2.9583333333333335
Ready to Wear (Pret-A-Porter) (1994)
2.8333333333333335
Cook the Thief His Wife & Her Lover, The (1989)
3.1363636363636362
Seven Years in Tibet (1997)
3.4285714285714284
Chasing Amy (1997)
3.576923076923077
Cruel Intentions (1999)
3.225
Final Destination (2000)
2.9285714285714284
Rules of Engagement (2000)
3.0714285714285716
Gone in 60 Seconds (2000)
3.221311475409836
Patriot, The (2000)
3.448529411764706
Space Cowboys (2000)


Over the Hedge (2006)
3.4285714285714284
Break-Up, The (2006)
2.590909090909091
Jesus Camp (2006)
4.071428571428571
Waitress (2007)
3.8333333333333335
Host, The (Gwoemul) (2006)
4.0
Meet the Robinsons (2007)
3.5454545454545454
How the Grinch Stole Christmas! (1966)
3.590909090909091
21 Up (1977)
4.0
Hairspray (2007)
3.5
Hostel: Part II (2007)
2.0
Broken Arrow (1950)
3.0
Charlie Bartlett (2007)
3.5
King of Kong, The (2007)
3.9166666666666665
Lust, Caution (Se, jie) (2007)
3.75
Dan in Real Life (2007)
3.388888888888889
Futurama: Bender's Big Score (2007)
3.9
Orphanage, The (Orfanato, El) (2007)
3.875
Maxed Out: Hard Times, Easy Credit and the Era of Predatory Lenders (2006)
3.0
Fool's Gold (2008)
2.3333333333333335
Step Up 2 the Streets (2008)
3.25
Penelope (2006)
3.5
Leatherheads (2008)
2.375
Bigger, Stronger, Faster* (2008)
3.5
Zombie Strippers! (2008)
0.5
Nick and Norah's Infinite Playlist (2008)
3.35
Duchess, The (2008)
3.5
Madagascar: Escape 2 Africa (2008)
3.6923076923076925
Away W

Girls Just Want to Have Fun (1985)
3.3333333333333335
Mummy: Tomb of the Dragon Emperor, The (2008)
2.642857142857143
W. (2008)
2.5
Punisher: War Zone (2008)
3.0
Peggy Sue Got Married (1986)
3.2142857142857144
Grapes of Wrath, The (1940)
4.111111111111111
How to Get Ahead in Advertising (1989)
3.625
Wait Until Dark (1967)
3.4285714285714284
Night of the Hunter, The (1955)
4.0
Goodbye, Mr. Chips (1939)
3.6666666666666665
Little Caesar (1931)
4.0
Stage Door (1937)
4.166666666666667
Silver Streak (1976)
3.6666666666666665
What the #$*! Do We Know!? (a.k.a. What the Bleep Do We Know!?) (2004)
2.75
Nobody Knows (Dare mo shiranai) (2004)
4.25
Mr. & Mrs. Smith (1941)
2.875
Adventures of Pluto Nash, The (2002)
2.3333333333333335
Soul Plane (2004)
3.75
First Beautiful Thing, The (La prima cosa bella) (2010)
4.0
Bullet to the Head (2012)
1.5
Rock of Ages (2012)
2.75
Immature, The (Immaturi) (2011)
3.5
Phil Spector (2013)
2.0
Bling Ring, The (2013)
2.0
Little Shop of Horrors, The (1960)
3.0
Tales

Big Mommas: Like Father, Like Son (2011)
1.0
African Cats (2011)
2.5
Inbetweeners Movie, The (2011)
3.6666666666666665
Red State (2011)
3.0
Shark Night 3D (2011)
1.5
Footloose (2011)
2.75
J. Edgar (2011)
2.5
Bunraku (2010)
1.5
Joyful Noise (2012)
1.5
Albatross (2011)
2.5
For a Good Time, Call... (2012)
2.5
Big Bang, The (2011)
1.5
Think Like a Man (2012)
2.5
Inhale (2010)
2.5
To Rome with Love (2012)
3.0
ParaNorman (2012)
3.5
Vamps (2012)
1.5
Fire with Fire (2012)
1.5
Parental Guidance (2012)
2.0
Beautiful Creatures (2013)
2.0
Brass Teapot, The (2012)
2.0
Host, The (2013)
2.5
42 (2013)
4.0
Yongary: Monster from the Deep (1967)
0.5
English Teacher, The (2013)
2.0
Behind the Candelabra (2013)
3.0
Not Suitable for Children (2012)
2.5
Lifeguard, The (2013)
3.5
Turbo (2013)
2.5
Revenge for Jolly! (2012)
2.0
Man of Tai Chi (2013)
3.25
Inescapable (2012)
1.0
Counselor, The (2013)
1.5
All is Bright (2013)
1.5
Bad Karma (2012)
2.0
Drift (2013)
2.0
Better Living Through Chemistry (2014)
1.5
Date